In [5]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatAnthropic
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.llms import Ollama

REDIS_URL = "redis://localhost:6379/0"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant who's good at {ability}. You always answer concisely, in under 20 words. Never prepend your responses with 'AI: `, just answer the question."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

model = Ollama(model="mistral")

chain = prompt | model

history = RedisChatMessageHistory(session_id="foo", url=REDIS_URL, ttl=600)

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: history,
    input_messages_key="question",
    history_messages_key="history",
)

response = chain_with_history.invoke(
    {"ability": "math", "question": "what are they good for?"},
    config={"configurable": {"session_id": "foobar"}},
)

In [6]:
print(response)

 A protractor is a tool used to measure angles.


In [8]:
history.messages

[]